# Import packages

In [2]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import timeseries_dataset_from_array

# Load MD trajectories

filename should be the location where the numpy array is stored 

In [3]:
# load cartesian coordinate file. 
file_name = '/Users/user/Mirror/long_seq_LSTM/extracted_pdb_trajectory_more_config_recentering_aligned.npy'
full_data = np.load(file_name)[:,1:]




In [4]:
# shape of the data 
full_data.shape, 'number of atoms: ', full_data.shape[1]/3


((5856, 6546), 'number of atoms: ', 2182.0)

# Select the training set and normalise 

In [5]:
training_set = full_data


training_set.shape

(5856, 6546)

Reshape the data to 2D and then normalise the three axis together

In [6]:
num_samples, num_features = training_set.shape
data_2d = training_set.reshape(num_samples * num_features, 1)



scaler = MinMaxScaler()
scaler.fit(data_2d)

data_2d = scaler.transform(data_2d)


training_set = data_2d.reshape(num_samples, num_features)


training_set.shape, f'max value: {training_set.max()} and the min valu {training_set.min()}'

((5856, 6546), 'max value: 0.9999999999999998 and the min valu 0.0')

In [7]:
features = training_set[:4000,:3]
target = training_set[1000:5000,:3]
features.shape, target.shape

((4000, 3), (4000, 3))

## Save the scaler to rescale the predictions later 

# Convert the data into timeSeries data 

In [8]:
def get_time_aromic_dataset(data_x,data_y,seq_length,t_func=timeseries_dataset_from_array,batch_num = 4000):
    x = t_func(
        data=data_x,
        targets=None,
        # sequence_stride = seq_length,
        sequence_stride = 1,
        sequence_length=seq_length,
        batch_size = batch_num
        
    )

    y = t_func(
        data=data_y,
        targets=None,
        # sequence_stride = seq_length,
        sequence_stride = 1,
        sequence_length=seq_length,
        batch_size = batch_num
    )


    
    for input in x:
        print(f'x shape: {input.shape}')
        break
        

    for input in y:
        print(f'y shape: {input.shape}')
        break

    
    for batch in zip(x, y):
        
        inputs, targets = batch
        
        break

    # targets = np.array(targets).reshape(targets.shape[0],(targets.shape[1]*targets.shape[2]))

    # print('target and features')
    # print(inputs.shape)
    # print(targets.shape)
    return inputs, targets

In [9]:
features,target = get_time_aromic_dataset(features,target,1000)

2023-12-19 02:12:42.523176: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2023-12-19 02:12:42.524223: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


x shape: (3001, 1000, 3)
y shape: (3001, 1000, 3)


In [10]:
features.shape

TensorShape([3001, 1000, 3])

# Test short lstm model

In [11]:
feature_dim = 3
sequence_length = 1000
lstm_dim = 64
BATCH_size = 500

# TensorFlow packages to train the lstm models


In [12]:
# Import tensorflow
from tensorflow import keras
import tensorflow as tf
# from AutoEncoder import AutoEncode_layer_build

from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Bidirectional, TimeDistributed, Concatenate
from tensorflow.keras import layers

# Define the lstm model and Compile it

In [32]:
inputs = Input(shape=(sequence_length, feature_dim))

x = layers.LSTM(lstm_dim, return_sequences=True )(inputs)

x = layers.LSTM(lstm_dim, return_sequences=True )(x)

output = layers.TimeDistributed(layers.Dense(3, activation='linear'))(x)


In [45]:
# discount_factor_input = Input(shape=(1))

# # Input for features
# features_input = Input(shape=(sequence_length, feature_dim))

# # Repeat discount factor along the sequence length
# discount_factor_sequence = layers.RepeatVector(sequence_length)(discount_factor_input)





# Input for features
features_input = Input(shape=(sequence_length, feature_dim), name='features_input')

# Input for discount factors
discount_factor_input = Input(shape=(sequence_length, 1), name='discount_factor_input')

# Merge features and discount factors along the last dimension
merged_input = Concatenate(axis=-1)([features_input, discount_factor_input])

# # Concatenate features and discount factor along the feature dimension
# # combined_input = Concatenate(axis=-1)([features_input, discount_factor_sequence])
# combined_input = Concatenate(axis=-1)([features_input, discount_factor_sequence])

# LSTM layers
x = LSTM(lstm_dim, return_sequences=True)(merged_input)
x = LSTM(lstm_dim, return_sequences=True)(x)

# TimeDistributed Dense layer for output
output = layers.TimeDistributed(Dense(3, activation='linear'))(x)

In [46]:
model = tf.keras.Model(inputs = [features_input, discount_factor_input], outputs = output)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features_input (InputLayer)     [(None, 1000, 3)]    0                                            
__________________________________________________________________________________________________
discount_factor_input (InputLay [(None, 1000, 1)]    0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 1000, 4)      0           features_input[0][0]             
                                                                 discount_factor_input[0][0]      
__________________________________________________________________________________________________
lstm_8 (LSTM)                   (None, 1000, 64)     17664       concatenate_3[0][0]        

In [47]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy','mae',keras.metrics.RootMeanSquaredError()])


## Reinforcement learning using discount factors 

In [25]:
#  Initial high discount factor for short-term focus
initial_discount_factor = 0.9

# Gradual decrease in discount factor to emphasize long-term rewards
discount_factors = np.linspace(initial_discount_factor, 0.5, sequence_length).reshape(-1, 1)



In [26]:
# training_data_discount_factors = np.random.uniform(0, 1, (sequence_length, 1))

In [27]:
discount_factors[:5,:5]

array([[0.9      ],
       [0.8995996],
       [0.8991992],
       [0.8987988],
       [0.8983984]])

In [19]:
num_samples

5856

In [29]:
discount_factors = np.tile(discount_factors, (3001, 1, 1))
discount_factors.shape

(3001, 1000, 1)

In [49]:
from tensorflow.keras.callbacks import LambdaCallback

# Define a function to print the discount factor during training
def print_discount_factor(epoch, logs, model, features, discount_factors):
    # Get the discount factor values from the discount_factors input data
    discount_factor_values = discount_factors[:, :, 0]  # Assuming discount_factors has shape (batch_size, sequence_length, 1)
    print("Current discount factor values:", discount_factor_values)

# Create a LambdaCallback to call the print_discount_factor function
discount_factor_callback = LambdaCallback(on_epoch_end=lambda epoch, logs: print_discount_factor(epoch, logs, model, features, discount_factors))


In [51]:
model.fit([features, discount_factors], 
          target, 
          epochs=100,
          callbacks=[discount_factor_callback],
          validation_split=0.2)

Epoch 1/100
75/75 [==============================] - 38s 498ms/step - loss: 0.0028 - accuracy: 0.8181 - mae: 0.0401 - root_mean_squared_error: 0.0528 - val_loss: 0.0044 - val_accuracy: 0.9434 - val_mae: 0.0504 - val_root_mean_squared_error: 0.0664
Current discount factor values: [[0.9       0.8995996 0.8991992 ... 0.5008008 0.5004004 0.5      ]
 [0.9       0.8995996 0.8991992 ... 0.5008008 0.5004004 0.5      ]
 [0.9       0.8995996 0.8991992 ... 0.5008008 0.5004004 0.5      ]
 ...
 [0.9       0.8995996 0.8991992 ... 0.5008008 0.5004004 0.5      ]
 [0.9       0.8995996 0.8991992 ... 0.5008008 0.5004004 0.5      ]
 [0.9       0.8995996 0.8991992 ... 0.5008008 0.5004004 0.5      ]]
Epoch 2/100
75/75 [==============================] - 36s 487ms/step - loss: 0.0025 - accuracy: 0.8437 - mae: 0.0373 - root_mean_squared_error: 0.0502 - val_loss: 0.0052 - val_accuracy: 0.9700 - val_mae: 0.0555 - val_root_mean_squared_error: 0.0721
Current discount factor values: [[0.9       0.8995996 0.8991992 

KeyboardInterrupt: 

In [31]:
target.shape

TensorShape([3001, 1000, 3])

In [33]:
features.shape

TensorShape([3001, 1000, 3])

In [34]:
discount_factors.shape

(3001, 1000, 1)

In [13]:
discount_factor_input = Input(shape=(1,))

# Input for features
features_input = Input(shape=(sequence_length, feature_dim))

# LSTM layers
x = LSTM(lstm_dim, return_sequences=True)(features_input)
x = LSTM(lstm_dim, return_sequences=True)(x)

# TimeDistributed Dense layer for output
# output = TimeDistributed(Dense(3, activation='linear'))(x)
output = layers.TimeDistributed(layers.Dense(3, activation='linear'))(x)
# Model
# model2 = tf.keras.Model(inputs=[features_input, discount_factor_input], outputs=output)
model2 = tf.keras.Model(inputs=features_input, outputs=output)

# Compile the model (adjust optimizer and loss as needed)
model2.compile(optimizer='adam', loss='mean_squared_error')

# Print model summary
model2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1000, 3)]         0         
_________________________________________________________________
lstm (LSTM)                  (None, 1000, 64)          17408     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000, 64)          33024     
_________________________________________________________________
time_distributed (TimeDistri (None, 1000, 3)           195       
Total params: 50,627
Trainable params: 50,627
Non-trainable params: 0
_________________________________________________________________


In [14]:
model2.fit(features, 
          target, 
          epochs=100,
          batch_size=100)

Epoch 1/100
 4/31 [==>...........................] - ETA: 28s - loss: 0.1543

KeyboardInterrupt: 

In [14]:
features.shape

TensorShape([3001, 1000, 3])